In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils.np_utils as ku 
import numpy as np 
import pandas as pd
from nltk import word_tokenize, ngrams
import contractions
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

Notes:

Model -
2 dense layers with 25 hidden units each
100 epochs resulted in 9% accuracy

Data -
truncated 50 rows of taylor swift lyrics 
stored as variable size ngrams of sequences for each line of each song

Results -
Largely nonsensical but it doesn't seem like there are any direct quotes from her songs

In [17]:
def create_model(predictors, label, max_sequence_len, total_words, num_epochs, hidden_units):
    
    model = Sequential()
#     model.add(Embedding(total_words, 32, input_length=max_sequence_len-1))
    model.add(Dense(units=hidden_units, activation='relu', input_dim=predictors.shape[1]))
    model.add(Dense(units=hidden_units, activation='relu', input_dim=predictors.shape[1]))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=num_epochs, verbose=1, callbacks=[earlystop])
    print(model.summary())
    return model 

In [3]:
def csv_to_lst(file):
    """
    Grabs the lyric section of the given csv and convert it to a string
    Parameters:
        file (CSV): file path where csv is located
    Returns:
        A list string representing all of the text section in the csv
    """
    lyrics_df = pd.read_csv(file) 
#     lyrics_df = lyrics_df

    lyrics_list = list(lyrics_df['lyrics'])


    return lyrics_list

In [4]:
def pre_process_all_lyrics(lyric_list):
    processed = []
    for lyrics in lyric_list:
#         print(lyrics)
        processed.append(pre_process_text(lyrics))
    return processed

In [5]:
def pre_process_line(line):
    EMBED = 'embed'
    word_list = word_tokenize(line)
    
   
    #index = word_list.index('Lyrics')
    #word_list = word_list[index + 1:]
    
    no_brackets_list = []
    is_inside = False
    punctuation_list = '''!()-[];:'"\,<>./?@#$%^&*_~”“’‘--...``'''''
    
    for word in word_list:
        if word == '[':
            is_inside = True
        if word == ']':
            is_inside = False
        else:
            if not is_inside:
                if word not in punctuation_list:
                    no_brackets_list.append(word.lower())

    if len(no_brackets_list) > 0:
        end_word = no_brackets_list[len(no_brackets_list)-1]   
        if EMBED in end_word:
#             print(end_word)
            embed_location = end_word.index('embed') 
            # all lyric genius data comes with word 'Embed' at the end
            substr = end_word[:embed_location]
            if len(substr) > 0:
                no_brackets_list[len(no_brackets_list)-1] = substr
        
        end_word = no_brackets_list[len(no_brackets_list)-1]  
        new_end_word = ''
        # also sometimes has a number before embed in last word
        for c in end_word:
            if c.isdigit():
                break
            else:
                new_end_word = new_end_word + c
        if len(end_word) > 0:
            no_brackets_list[len(no_brackets_list)-1] = new_end_word
            
    ret = []
    for word in no_brackets_list:
        if len(word) < 0:
            break
        #idk why this is like this 
        elif word == "''":
            break
        else:
            ret.append(word)
    return ret

In [6]:
def pre_process_text(lyrics):
    """
    Preprocesses the text to add start and end tokens to each sentence
    Parameters:
        lst (list): list of lyrics 
    Returns:
        List of List of words with start and end tokens
    """
    
    lyrics = contractions.fix(lyrics)
#     print(lyrics)
    line_list = lyrics.split("\n")
    line_list = line_list[1:]
    
    ret = []
    for line in line_list:
        if not len(line) == 0:
            to_be_appended = pre_process_line(line)
            if len(to_be_appended) > 0:
                ret.append(to_be_appended)
    return ret

In [7]:
def dataset_preparation(data):

    # basic cleanup
#     corpus = data.lower().split("\n")
    # tokenization	
    corpus = flatten_lines(data)
    print(corpus[:10])
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    print('tokenization:')
    print(tokenizer.word_index)
    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    print('input_sequences:')
    print(input_sequences)
    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    print('padded input_sequences:')
    print(input_sequences)
    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    
    print('X predictors:')
    print(predictors)
    print('y labels:')
    print(label)
    label = ku.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len, total_words

In [8]:
def flatten_lines(data):
    corpus = []
    for song in data:
        for line in song:
            line_string = ""
            for word in line: 
                line_string = line_string + " " + word
            corpus.append(line_string)
    return corpus

In [9]:
lst = csv_to_lst('taylorswift.csv')
data = pre_process_all_lyrics(lst)[:50]
# print(data)

In [78]:
# for song in data:
#     for line in song:
#         print(line)

In [10]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)

[' i walked through the door with you the air was cold', " but something 'bout it felt like home somehow", " and i left my scarf there at your sister 's house", ' and you have still got it in your drawer even now', ' oh your sweet disposition and my wide-eyed gaze', ' we are singin in the car getting lost upstate', ' autumn leaves fallin down like pieces into place', ' and i can picture it after all these days', ' and i know it is long gone and', " that magic 's not here no more"]
tokenization:
{'i': 1, 'you': 2, 'the': 3, 'it': 4, 'and': 5, 'me': 6, 'to': 7, 'my': 8, 'a': 9, 'is': 10, 'not': 11, 'in': 12, 'your': 13, 'of': 14, 'was': 15, 'that': 16, 'but': 17, 'all': 18, 'oh': 19, 'do': 20, 'like': 21, 'be': 22, 'we': 23, 'am': 24, 'are': 25, 'there': 26, 'so': 27, 'just': 28, 'what': 29, 'for': 30, 'have': 31, 'would': 32, 'know': 33, 'on': 34, 'can': 35, 'will': 36, 'want': 37, 'because': 38, 'never': 39, 'this': 40, 'if': 41, 'with': 42, 'when': 43, 'they': 44, 'now': 45, 'time': 4

In [11]:
print('predictors: ', len(predictors))
# print(predictors[:3])
print('labels: ', label.shape)
# print(label[:3])
print('max sequence len: ', max_sequence_len)
print('total words: ', total_words)

predictors:  19137
labels:  (19137, 2090)
max sequence len:  19
total words:  2090


In [20]:
model = create_model(predictors, label, max_sequence_len, total_words, 100, 25)

Epoch 1/100
599/599 [==============================] - 1s 2ms/step - loss: 8.0837 - accuracy: 0.0397
Epoch 2/100
599/599 [==============================] - 1s 2ms/step - loss: 6.4247 - accuracy: 0.0484
Epoch 3/100
599/599 [==============================] - 1s 1ms/step - loss: 6.0415 - accuracy: 0.0494
Epoch 4/100
599/599 [==============================] - 1s 1ms/step - loss: 5.8373 - accuracy: 0.0527
Epoch 5/100
599/599 [==============================] - 1s 2ms/step - loss: 5.6955 - accuracy: 0.0543
Epoch 6/100
599/599 [==============================] - 1s 2ms/step - loss: 5.6304 - accuracy: 0.0546
Epoch 7/100
599/599 [==============================] - 1s 2ms/step - loss: 5.4939 - accuracy: 0.0559
Epoch 8/100
599/599 [==============================] - 1s 2ms/step - loss: 5.3885 - accuracy: 0.0574
Epoch 9/100
599/599 [==============================] - 1s 2ms/step - loss: 5.3149 - accuracy: 0.0582
Epoch 10/100
599/599 [==============================] - 1s 2ms/step - loss: 5.2577 - accura

599/599 [==============================] - 1s 1ms/step - loss: 4.7952 - accuracy: 0.0745
Epoch 28/100
599/599 [==============================] - 1s 2ms/step - loss: 4.8186 - accuracy: 0.0738
Epoch 29/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7869 - accuracy: 0.0762
Epoch 30/100
599/599 [==============================] - 1s 2ms/step - loss: 4.8187 - accuracy: 0.0764
Epoch 31/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7776 - accuracy: 0.0757
Epoch 32/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7480 - accuracy: 0.0760
Epoch 33/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7369 - accuracy: 0.0789
Epoch 34/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7707 - accuracy: 0.0763
Epoch 35/100
599/599 [==============================] - 1s 2ms/step - loss: 4.7258 - accuracy: 0.0768
Epoch 36/100
599/599 [==============================] - 1s 1ms/step - loss: 4.7151 - accuracy: 

599/599 [==============================] - 1s 1ms/step - loss: 4.6364 - accuracy: 0.0879
Epoch 54/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6416 - accuracy: 0.0905
Epoch 55/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6220 - accuracy: 0.0883
Epoch 56/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6535 - accuracy: 0.0860
Epoch 57/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6451 - accuracy: 0.0897
Epoch 58/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6330 - accuracy: 0.0909
Epoch 59/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6424 - accuracy: 0.0911
Epoch 60/100
599/599 [==============================] - 1s 1ms/step - loss: 4.8044 - accuracy: 0.0828
Epoch 61/100
599/599 [==============================] - 1s 1ms/step - loss: 4.7071 - accuracy: 0.0863
Epoch 62/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6412 - accuracy: 

599/599 [==============================] - 1s 1ms/step - loss: 4.5813 - accuracy: 0.0931
Epoch 80/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5830 - accuracy: 0.0938
Epoch 81/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6282 - accuracy: 0.0946
Epoch 82/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5873 - accuracy: 0.0958
Epoch 83/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5907 - accuracy: 0.0971
Epoch 84/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5549 - accuracy: 0.0966
Epoch 85/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6175 - accuracy: 0.0975
Epoch 86/100
599/599 [==============================] - 1s 1ms/step - loss: 4.6045 - accuracy: 0.0968
Epoch 87/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5602 - accuracy: 0.0956
Epoch 88/100
599/599 [==============================] - 1s 1ms/step - loss: 4.5831 - accuracy: 

In [21]:
def generate_text(tokenizer, seed_text, next_words, max_sequence_len):
#     lines = []
    LINE_LEN = np.random.randint(1, max_sequence_len)
    words = []
    words.extend(seed_text.split())
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
#         print('-----')
#         print(predicted.shape)
#         choice = np.argmax(predicted)
        choice = np.random.choice(len(predicted[0]), p=predicted[0])
#         choice = sample(predicted[0])
#         print('choice: ', choice)
#         print(index_of)
        word = tokenizer.index_word[choice]
        words.append(word)
#         print('word: ', word)
        if (len(words) % LINE_LEN == 0): # new line time
#             print(words)
#             print(len(words))
            words.append('\n')
            seed_text = word
        else:
            seed_text = seed_text + " " + word
#         seed_text = word
#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted[0]:
#                 output_word = word
#                 break
#         seed_text += " " + output_word
    return format_lyrics(words)

In [22]:
def sample(preds, temperature=1):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [23]:
def format_lyrics(words):
    lyr = ""
    for word in words:
        lyr = lyr + " " + word
    return lyr

In [24]:
generated = generate_text(tokenizer, "you", 50, max_sequence_len)
print(generated)

 you are you want me hurt back time being 
 was one can man there boy space pack 
 fits i you taylor the things it you 
 will shit in i weekend like my life 
 reputation that into your take my it peace 
 aids land skies cold better swearing or in 
 baby me


In [25]:
generated2 = generate_text(tokenizer, "i", 50, max_sequence_len)
print(generated2)

 i got they been walking out door the take yeah live 
 time he bad of angry closer my man now mouth 
 flaw and the chosen hope to door hell red all 
 the never wasted on fate down i would beautiful home 
 knows me were borrowed them to pretty first so scarf 



In [26]:
generated3 = generate_text(tokenizer, "this is", 50, max_sequence_len)
print(generated3)

 this is the to 
 not this not 
 mom's with i 
 woke plan and 
 i planned i 
 know one a 
 eclipsed meetings i 
 knew car this 
 everyone in for 
 take me is 
 you say i 
 know be how 
 them been reputation 
 you was love 
 there that same 
 all time i 
 know i high 



In [27]:
generated4 = generate_text(tokenizer, "his", 50, max_sequence_len)
print(generated4)

 his when tell so want peace place to wishing there is home my in thing air 
 like are of time very blood you just first never i go oh type it 
 was cryin is you baller evergreen my are losin all born some red back can 
 say the all hindsight ocean


In [28]:
generated5 = generate_text(tokenizer, "my", 50, max_sequence_len)
print(generated5)

 my i are on drinkin 
 to the blue go 
 could not just made 
 me reminds take barren 
 wants i you know 
 still better walked you 
 are you are light 
 that sinking just i 
 know a rainbow my 
 if i did to 
 you were not the 
 world a my for 
 mystical motion
